## Note about cuDF DataFrames/Series

A [DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) is a collection of [Series](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html); The DataFrame is the way CuDF represents a table, and Series is the data-structure CuDF uses to represent a column.

What makes CuDF so attractive is the powerful interface to access individual records of the table, proper handling of missing values, and relational-databases operations between DataFrames.

## Selecting values

To access a value at the position `[i,j]` of a DataFrame, we have two options, depending on what is the meaning of `i` in use. Remember that a DataFrame provides a *index* as a way to identify the rows of the table; a row, then, has a *position* inside the table as well as a *label*, which uniquely identifies its *entry* in the DataFrame.

## Use `DataFrame.iloc[..., ...]` to select values by their (entry) position

* Can specify location by numerical index analogously to 2D version of character selection in strings.

In [1]:
import cudf
data = cudf.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.iloc[0, 0])

gdpPercap_1952    1601.056136
Name: Albania, dtype: float64


## Use `DataFrame.loc[..., ...]` to select values by their (entry) label.

* Can specify location by row name analogously to 2D version of dictionary keys.

In [2]:
data = cudf.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.loc["Albania", "gdpPercap_1952"])

gdpPercap_1952    1601.056136
Name: Albania, dtype: float64


## Use `:` on its own to mean all columns or all rows.

* Just like Python's usual slicing notation.

In [3]:
print(data.loc["Albania", :])

gdpPercap_1952    1601.056136
gdpPercap_1957    1942.284244
gdpPercap_1962    2312.888958
gdpPercap_1967    2760.196931
gdpPercap_1972    3313.422188
gdpPercap_1977    3533.003910
gdpPercap_1982    3630.880722
gdpPercap_1987    3738.932735
gdpPercap_1992    2497.437901
gdpPercap_1997    3193.054604
gdpPercap_2002    4604.211737
gdpPercap_2007    5937.029526
Name: Albania, dtype: float64


* Would get the same result printing `data.loc["Albania"]` (without a second index).

In [4]:
print(data.loc[:, "gdpPercap_1952"])

Albania                    1601.056136
Austria                    6137.076492
Belgium                    8343.105127
Bosnia and Herzegovina      973.533195
Bulgaria                   2444.286648
Croatia                    3119.236520
Czech Republic             6876.140250
Denmark                    9692.385245
Finland                    6424.519071
France                     7029.809327
Germany                    7144.114393
Greece                     3530.690067
Hungary                    5263.673816
Iceland                    7267.688428
Ireland                    5210.280328
Italy                      4931.404155
Montenegro                 2647.585601
Netherlands                8941.571858
Norway                    10095.421720
Poland                     4029.329699
Portugal                   3068.319867
Romania                    3144.613186
Serbia                     3581.459448
Slovak Republic            5074.659104
Slovenia                   4215.041741
Spain                    

* Would get the same result printing `data["gdpPercap_1952"]`
* Also get the same result printing `data.gdpPercap_1952` (since it's a column name)

## Select multiple columns or rows using `DataFrame.loc` and a named slice.

In [5]:
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'])

             gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
Italy           8243.582340    10022.401310    12269.273780
Montenegro      4649.593785     5907.850937     7778.414017
Netherlands    12790.849560    15363.251360    18794.745670
Norway         13450.401510    16361.876470    18965.055510
Poland          5338.752143     6557.152776     8006.506993


In the above code, we discover that **slicing using `loc` is inclusive at both ends**, which differs from **slicing using `iloc`**, where slicing indicates everything up to but not including the final index.

## Result of slicing can be used in further operations.

* Usually don't just print a slice.
* All the statistical operators that work on entire dataframes work the same way on slices.
* E.g., calculate max of a slice.

In [6]:
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'].max())

gdpPercap_1962    13450.40151
gdpPercap_1967    16361.87647
gdpPercap_1972    18965.05551
dtype: float64


In [7]:
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'].min())

gdpPercap_1962    4649.593785
gdpPercap_1967    5907.850937
gdpPercap_1972    7778.414017
dtype: float64


## Use comparisons to select data based on value.

* Comparison is applied element by element.
* Returns a similarly-shaped dataframe of `True` and `False`.

In [8]:
# Use a subset of data to keep output readable.
subset = data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972']
print('Subset of data:\n', subset)

# Which values were greater than 10000 ?
print('\nWhere are values large?\n', subset > 10000)

Subset of data:
              gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
Italy           8243.582340    10022.401310    12269.273780
Montenegro      4649.593785     5907.850937     7778.414017
Netherlands    12790.849560    15363.251360    18794.745670
Norway         13450.401510    16361.876470    18965.055510
Poland          5338.752143     6557.152776     8006.506993

Where are values large?
              gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
Italy                 False            True            True
Montenegro            False           False           False
Netherlands            True            True            True
Norway                 True            True            True
Poland                False           False           False


## Select values or `NaN` using a Boolean mask.

* A frame full of Booleans is sometimes called a *mask* because of how it can be used.

In [9]:
mask = subset > 10000
print(subset[mask])

            gdpPercap_1962 gdpPercap_1967 gdpPercap_1972
Italy                 null    10022.40131    12269.27378
Montenegro            null           null           null
Netherlands    12790.84956    15363.25136    18794.74567
Norway         13450.40151    16361.87647    18965.05551
Poland                null           null           null


* Get the value where the mask is true, and NaN (Not a Number) where it is false.
* Useful because NaNs are ignored by operations like max, min, average, etc.

In [10]:
print(subset[subset > 10000].describe())

       gdpPercap_1962  gdpPercap_1967  gdpPercap_1972
count        2.000000        3.000000        3.000000
mean     13120.625535    13915.843047    16676.358320
std        466.373656     3408.589070     3817.597015
min      12790.849560    10022.401310    12269.273780
25%      12955.737548    12692.826335    15532.009725
50%      13120.625535    15363.251360    18794.745670
75%      13285.513523    15862.563915    18879.900590
max      13450.401510    16361.876470    18965.055510


## Differences Between cuDF and Pandas

Although cuDF is similar to Pandas, the signature of some cuDF functions differs from their Pandas equivalents.  Also, some Pandas methods and functions are completely missing from cuDF.  Here's a partial list of missing functionality. 

> - The apply() method is not available in cuDF
> - The aggregate() method is not available in cuDF
> - The info() method is not available in cuDF


## Questions

#### Q1: Selection of Individual Values

Assume cuDF has been imported into your notebook and the Gapminder GDP data for Europe has been loaded:

In [ ]:
import cudf

df = cudf.read_csv('data/gapminder_gdp_europe.csv', index_col='country')

Write an expression to find the Per Capita GDP of Serbia in 2007.

##### Solution

In [ ]:
%load solutions/071_solution_01.py

#### Q2: Extent of Slicing

1. Do the two statements below produce the same output?
2. Based on this, what rule governs what is included (or not) in numerical slices and named slices in Pandas?


In [ ]:
print(data.iloc[0:2, 0:2])
print(data.loc['Albania':'Belgium', 'gdpPercap_1952':'gdpPercap_1962'])

##### Solution

In [ ]:
%load solutions/071_solution_02.py

#### Q3: Reconstructing Data

Explain what each line in the following short program does: what is in first, second, etc.?

In [ ]:
Explain what each line in the following short program does: what is in first, second, etc.?

first = pandas.read_csv('data/gapminder_all.csv', index_col='country')
second = first[first['continent'] == 'Americas']
third = second.drop('Puerto Rico')
fourth = third.drop('continent', axis = 1)
fourth.to_csv('result.csv')

##### Solution

In [ ]:
%load solutions/071_solution_03.py

#### Q4: Selecting Indices

Explain in simple terms what idxmin and idxmax do in the short program below. When would you use these methods?

In [ ]:
data = cudf.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.idxmin())
print(data.idxmax())

##### Solution

In [ ]:
%load solutions/071_solution_04.py

#### Q5: Practice with Selection

Assume Pandas has been imported and the Gapminder GDP data for Europe has been loaded. Write an expression to select each of the following:

1. GDP per capita for all countries in 1982.
2. GDP per capita for Denmark for all years.
3. GDP per capita for all countries for years after 1985.
4. GDP per capita for each country in 2007 as a multiple of GDP per capita for that country in 1952.


##### Solution

In [1]:
%load solutions/071_solution_05.py

#### Q6: Using `dir` function to see available methods

Python includes a `dir` function that can be used to display all of the available methods (functions) that are built into a data object.  As an example, the  functions available for a [list data type](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists) are:

In [ ]:
potatoes = ["Russet", "Norkota", "Yukon Gold", "Pontiac"]
dir(potatoes)

This command returns:

In [ ]:
['__add__',
...
'__subclasshook__',
'append',
'clear',
'copy',
'count',
'extend',
'index',
'insert',
'pop',
'remove',
'reverse',
'sort']

The double underscore functions can be ignored for now; functions that are not surrounded by double underscores are the *public interface* of the [list type](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists). So, if you want to sort the list of potatoes, according to `dir` you should try,

In [ ]:
potatoes.sort()

Assume cuDF has been imported and the Gapminder GDP data for Europe has been loaded as `data`.  Then, use `dir` to find the function that prints out the median per-capita GDP across all European countries for each year that information is available.  

##### Solution

In [2]:
%load solutions/071_solution_06.py